In [16]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.vectorstores.faiss import FAISS
from langchain_core.documents import Document
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chat_models.gigachat import GigaChat
from langchain.chains import create_retrieval_chain
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_gigachat.chat_models import GigaChat
from langchain_gigachat.embeddings.gigachat import GigaChatEmbeddings
from pathlib import Path

In [17]:
API_KEY = "YjllY2FhYjgtNGRlMC00MDA4LWIwZmYtNjdlNjY0ZmI5OTc4OmRkMjZhOWFjLThhNTctNGM3ZC1iZjFkLWQ3NGY1NmRjNTQzMQ=="

In [18]:
doc_files = list(Path("..\\data\\documentation").rglob("*.md"))
code_files = list(Path("..\\data\\code").rglob("*.py"))

doc_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
code_splitter = RecursiveCharacterTextSplitter(separators=["\nclass", "\ndef", "\n"], chunk_size=500, chunk_overlap=100)

doc_chunks = [doc_splitter.split_text(Path(file).read_text(encoding="utf-8")) for file in doc_files]
code_chunks = [code_splitter.split_text(Path(file).read_text(encoding="utf-8")) for file in code_files]

docs = [Document(page_content=chunk, metadata={"source": "doc"}) for file in doc_files for chunk in doc_splitter.split_text(Path(file).read_text(encoding="utf-8"))]
code = [Document(page_content=chunk, metadata={"source": "code"}) for file in code_files for chunk in code_splitter.split_text(Path(file).read_text(encoding="utf-8"))]

documents = docs + code

In [19]:
embedding=GigaChatEmbeddings(
        credentials=API_KEY,
        scope="GIGACHAT_API_PERS",
        verify_ssl_certs=False,
)

vector_store = FAISS.from_documents(docs, embedding=embedding)

In [20]:
retriever = vector_store.as_retriever(search_kwargs={"k": 5})

In [25]:
llm = GigaChat(
    credentials=API_KEY,
    verify_ssl_certs=False,
)

prompt = ChatPromptTemplate.from_template('''Ты — технический помощник, работающий с библиотекой LangChain. У тебя есть доступ к документации и к исходному коду библиотеки.

Твоя задача — ответить на вопрос пользователя, используя и документацию, и код. Следуй этим правилам:

---

1. 📄 Если информация взята из документации, укажи, из какого именно документа она была получена. Пример:  
   _"Согласно документации (load_chain.md)..."_

2. 🧩 Если информация взята из исходного кода, укажи, из какого файла она была получена. Пример:  
   _"В коде (loader.py) реализована только цепочка summarize_chain..."_

3. ⚠️ Если документация и код противоречат друг другу, всё равно сформулируй полезный ответ, но обязательно предупреди об этом. Пример:  
   _"Документация (load_chain.md) утверждает, что поддерживаются 3 цепочки, однако в коде (loader.py) реализована только одна. Это потенциальное несоответствие."_

4. ❓ Если ты не уверен, соответствует ли документация коду, также предупреди об этом. Пример:  
   _"Не удалось однозначно проверить, соответствует ли описание в документации (agent_overview.md) текущей реализации кода (agent/base.py). Будьте внимательны."_

5. 💬 Избегай вымышленных деталей — все факты должны быть подтверждены фрагментами из кода или документации.

---

В конце ответа сделай краткое заключение:
- Указан ли источник каждого утверждения?
- Есть ли возможные противоречия?

---

Теперь ответь на вопрос:
"{context}"'''
)

In [26]:
document_chain = create_stuff_documents_chain(
    llm=llm,
    prompt=prompt
)

retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [27]:
q1 = 'What is LangSmith?'

resp1 = retrieval_chain.invoke(
    {'input': q1}
)

resp1

{'input': 'What is LangSmith?',
 'context': [Document(id='51d527a9-9ec0-4383-bc25-2a187cb38bae', metadata={'source': 'doc'}, page_content='## Обзор агента\n\nАгент LangChain использует инструментальную среду и LLM для принятия решений о действиях. Он может вызывать внешние API и использовать инструменты.'),
  Document(id='25afbb9a-6f6d-49f1-b889-d909cc7a5468', metadata={'source': 'doc'}, page_content='## Векторные хранилища\n\nLangChain поддерживает FAISS, Chroma, Weaviate, Pinecone и другие. Для загрузки используется метод `from_documents`.'),
  Document(id='bedeed30-de04-4f7c-88c5-0000a88f4ac9', metadata={'source': 'doc'}, page_content='## Чат-модели\n\nLangChain интегрирован с OpenAI, Anthropic, Azure. Чат-модель вызывается через `ChatOpenAI`, `ChatAnthropic` и т.п.'),
  Document(id='2b597f94-7c54-47e5-a85c-24019fd4d450', metadata={'source': 'doc'}, page_content='## Обработка ошибок\n\nМногие классы LangChain выбрасывают собственные ошибки, например `OutputParserException`, `ChainLo

In [28]:
q1 = 'Как агент принимает решение, что делать?'

resp1 = retrieval_chain.invoke(
    {'input': q1}
)

resp1

{'input': 'Как агент принимает решение, что делать?',
 'context': [Document(id='51d527a9-9ec0-4383-bc25-2a187cb38bae', metadata={'source': 'doc'}, page_content='## Обзор агента\n\nАгент LangChain использует инструментальную среду и LLM для принятия решений о действиях. Он может вызывать внешние API и использовать инструменты.'),
  Document(id='9dbb8b6b-6c12-4517-8282-df40eab05078', metadata={'source': 'doc'}, page_content='## Инструменты агента\n\nИнструменты позволяют агенту выполнять задачи: делать HTTP-запросы, выполнять вычисления, обращаться к БД и т.д.'),
  Document(id='db6f70f1-c70b-4982-8dda-76829bdda0dd', metadata={'source': 'doc'}, page_content='## Пользовательские цепочки\n\nМожно определить свои цепочки, наследуясь от класса `Chain` и реализуя методы `__init__` и `run`.'),
  Document(id='bedeed30-de04-4f7c-88c5-0000a88f4ac9', metadata={'source': 'doc'}, page_content='## Чат-модели\n\nLangChain интегрирован с OpenAI, Anthropic, Azure. Чат-модель вызывается через `ChatOpenAI`

In [29]:
q1 = 'Какие векторные базы поддерживает LangChain?'

resp1 = retrieval_chain.invoke(
    {'input': q1}
)

resp1

{'input': 'Какие векторные базы поддерживает LangChain?',
 'context': [Document(id='25afbb9a-6f6d-49f1-b889-d909cc7a5468', metadata={'source': 'doc'}, page_content='## Векторные хранилища\n\nLangChain поддерживает FAISS, Chroma, Weaviate, Pinecone и другие. Для загрузки используется метод `from_documents`.'),
  Document(id='bedeed30-de04-4f7c-88c5-0000a88f4ac9', metadata={'source': 'doc'}, page_content='## Чат-модели\n\nLangChain интегрирован с OpenAI, Anthropic, Azure. Чат-модель вызывается через `ChatOpenAI`, `ChatAnthropic` и т.п.'),
  Document(id='2b597f94-7c54-47e5-a85c-24019fd4d450', metadata={'source': 'doc'}, page_content='## Обработка ошибок\n\nМногие классы LangChain выбрасывают собственные ошибки, например `OutputParserException`, `ChainLoadError` и другие.'),
  Document(id='51d527a9-9ec0-4383-bc25-2a187cb38bae', metadata={'source': 'doc'}, page_content='## Обзор агента\n\nАгент LangChain использует инструментальную среду и LLM для принятия решений о действиях. Он может вызы

In [30]:
q1 = 'Что произойдет, если передать неизвестную цепочку в load_chain?'

resp1 = retrieval_chain.invoke(
    {'input': q1}
)

resp1

{'input': 'Что произойдет, если передать неизвестную цепочку в load_chain?',
 'context': [Document(id='6cb6439d-8850-4653-87c4-11388a941837', metadata={'source': 'doc'}, page_content='## Функция `load_chain`\n\nФункция `load_chain` предназначена для загрузки заранее настроенных цепочек обработки из конфигурационного файла или по имени.\n\n### Пример использования\n\n```python\nfrom langchain.chains import load_chain\n\nchain = load_chain("summarize_chain")\nresponse = chain.run("This is a long article text...")'),
  Document(id='db6f70f1-c70b-4982-8dda-76829bdda0dd', metadata={'source': 'doc'}, page_content='## Пользовательские цепочки\n\nМожно определить свои цепочки, наследуясь от класса `Chain` и реализуя методы `__init__` и `run`.'),
  Document(id='2b597f94-7c54-47e5-a85c-24019fd4d450', metadata={'source': 'doc'}, page_content='## Обработка ошибок\n\nМногие классы LangChain выбрасывают собственные ошибки, например `OutputParserException`, `ChainLoadError` и другие.'),
  Document(id

In [31]:
q1 = 'Как реализовать собственную цепочку?'

resp1 = retrieval_chain.invoke(
    {'input': q1}
)

resp1

{'input': 'Как реализовать собственную цепочку?',
 'context': [Document(id='db6f70f1-c70b-4982-8dda-76829bdda0dd', metadata={'source': 'doc'}, page_content='## Пользовательские цепочки\n\nМожно определить свои цепочки, наследуясь от класса `Chain` и реализуя методы `__init__` и `run`.'),
  Document(id='6cb6439d-8850-4653-87c4-11388a941837', metadata={'source': 'doc'}, page_content='## Функция `load_chain`\n\nФункция `load_chain` предназначена для загрузки заранее настроенных цепочек обработки из конфигурационного файла или по имени.\n\n### Пример использования\n\n```python\nfrom langchain.chains import load_chain\n\nchain = load_chain("summarize_chain")\nresponse = chain.run("This is a long article text...")'),
  Document(id='2b597f94-7c54-47e5-a85c-24019fd4d450', metadata={'source': 'doc'}, page_content='## Обработка ошибок\n\nМногие классы LangChain выбрасывают собственные ошибки, например `OutputParserException`, `ChainLoadError` и другие.'),
  Document(id='25afbb9a-6f6d-49f1-b889-d

In [32]:
q1 = 'Как использовать PromptTemplate?'

resp1 = retrieval_chain.invoke(
    {'input': q1}
)

resp1

{'input': 'Как использовать PromptTemplate?',
 'context': [Document(id='bedeed30-de04-4f7c-88c5-0000a88f4ac9', metadata={'source': 'doc'}, page_content='## Чат-модели\n\nLangChain интегрирован с OpenAI, Anthropic, Azure. Чат-модель вызывается через `ChatOpenAI`, `ChatAnthropic` и т.п.'),
  Document(id='25afbb9a-6f6d-49f1-b889-d909cc7a5468', metadata={'source': 'doc'}, page_content='## Векторные хранилища\n\nLangChain поддерживает FAISS, Chroma, Weaviate, Pinecone и другие. Для загрузки используется метод `from_documents`.'),
  Document(id='7a9b4c52-88d6-4cc9-a314-2c6f7cf2eaba', metadata={'source': 'doc'}, page_content='## Загрузка документов\n\nЗагрузка поддерживается с помощью `DirectoryLoader`, `WebBaseLoader`, `PDFLoader` и т.д.'),
  Document(id='db6f70f1-c70b-4982-8dda-76829bdda0dd', metadata={'source': 'doc'}, page_content='## Пользовательские цепочки\n\nМожно определить свои цепочки, наследуясь от класса `Chain` и реализуя методы `__init__` и `run`.'),
  Document(id='51d527a9-9ec

In [33]:
q1 = 'Приведи пример загрузки файлов из директории'

resp1 = retrieval_chain.invoke(
    {'input': q1}
)

resp1

{'input': 'Приведи пример загрузки файлов из директории',
 'context': [Document(id='7a9b4c52-88d6-4cc9-a314-2c6f7cf2eaba', metadata={'source': 'doc'}, page_content='## Загрузка документов\n\nЗагрузка поддерживается с помощью `DirectoryLoader`, `WebBaseLoader`, `PDFLoader` и т.д.'),
  Document(id='6cb6439d-8850-4653-87c4-11388a941837', metadata={'source': 'doc'}, page_content='## Функция `load_chain`\n\nФункция `load_chain` предназначена для загрузки заранее настроенных цепочек обработки из конфигурационного файла или по имени.\n\n### Пример использования\n\n```python\nfrom langchain.chains import load_chain\n\nchain = load_chain("summarize_chain")\nresponse = chain.run("This is a long article text...")'),
  Document(id='25afbb9a-6f6d-49f1-b889-d909cc7a5468', metadata={'source': 'doc'}, page_content='## Векторные хранилища\n\nLangChain поддерживает FAISS, Chroma, Weaviate, Pinecone и другие. Для загрузки используется метод `from_documents`.'),
  Document(id='9dbb8b6b-6c12-4517-8282-df40

In [34]:
q1 = 'Какие ошибки бывают в LangChain и когда они возникают?'

resp1 = retrieval_chain.invoke(
    {'input': q1}
)

resp1

{'input': 'Какие ошибки бывают в LangChain и когда они возникают?',
 'context': [Document(id='2b597f94-7c54-47e5-a85c-24019fd4d450', metadata={'source': 'doc'}, page_content='## Обработка ошибок\n\nМногие классы LangChain выбрасывают собственные ошибки, например `OutputParserException`, `ChainLoadError` и другие.'),
  Document(id='25afbb9a-6f6d-49f1-b889-d909cc7a5468', metadata={'source': 'doc'}, page_content='## Векторные хранилища\n\nLangChain поддерживает FAISS, Chroma, Weaviate, Pinecone и другие. Для загрузки используется метод `from_documents`.'),
  Document(id='bedeed30-de04-4f7c-88c5-0000a88f4ac9', metadata={'source': 'doc'}, page_content='## Чат-модели\n\nLangChain интегрирован с OpenAI, Anthropic, Azure. Чат-модель вызывается через `ChatOpenAI`, `ChatAnthropic` и т.п.'),
  Document(id='51d527a9-9ec0-4383-bc25-2a187cb38bae', metadata={'source': 'doc'}, page_content='## Обзор агента\n\nАгент LangChain использует инструментальную среду и LLM для принятия решений о действиях. Он 